In [10]:
import pandas as pd
from nba_api.stats.endpoints import commonteamroster
from matplotlib import pyplot as plt
import numpy as np 
import datetime
import os
from nba_api.stats.static import players
import time
from typing import List, Iterable, Set, Dict, Optional
from nba_api.stats.endpoints import CommonTeamRoster

from glob import glob



In [3]:
df = pd.read_parquet('df_c0.parquet')
df

,SEASON_YEAR,GAME_ID,GAME_DATE,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,MATCHUP,WL,...,STL,BLK,TOV,PF,PFD,PLUS_MINUS,DD2,TD3,OPP_TEAM_ABBREVIATION,OPP_TEAM_ID
0,2025-26,0022500001,2025-10-21,1630598,Aaron Wiggins,1610612760,OKC,Oklahoma City Thunder,OKC vs. HOU,W,...,0,0,1,1,0,2,0,0,HOU,1610612745
1,2025-26,0022500001,2025-10-21,1628983,Shai Gilgeous-Alexander,1610612760,OKC,Oklahoma City Thunder,OKC vs. HOU,W,...,2,2,3,2,10,3,0,0,HOU,1610612745
2,2025-26,0022500001,2025-10-21,1631096,Chet Holmgren,1610612760,OKC,Oklahoma City Thunder,OKC vs. HOU,W,...,0,0,3,6,3,-1,0,0,HOU,1610612745
3,2025-26,0022500001,2025-10-21,1629652,Luguentz Dort,1610612760,OKC,Oklahoma City Thunder,OKC vs. HOU,W,...,1,0,1,4,2,2,0,0,HOU,1610612745
4,2025-26,0022500001,2025-10-21,1628392,Isaiah Hartenstein,1610612760,OKC,Oklahoma City Thunder,OKC vs. HOU,W,...,2,0,1,6,5,9,0,0,HOU,1610612745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,2025-26,0022500527,2026-01-07,1631126,Caleb Love,1610612757,POR,Portland Trail Blazers,POR vs. HOU,W,...,0,0,2,1,0,5,0,0,HOU,1610612745
173,2025-26,0022500527,2026-01-07,1642270,Donovan Clingan,1610612757,POR,Portland Trail Blazers,POR vs. HOU,W,...,0,1,0,0,0,-1,0,0,HOU,1610612745
174,2025-26,0022500527,2026-01-07,1631101,Shaedon Sharpe,1610612757,POR,Portland Trail Blazers,POR vs. HOU,W,...,1,0,3,2,6,-3,0,0,HOU,1610612745
175,2025-26,0022500527,2026-01-07,1641739,Toumani Camara,1610612757,POR,Portland Trail Blazers,POR vs. HOU,W,...,2,2,1,6,4,10,0,0,HOU,1610612745


In [4]:
STAT_COLS = ["MIN", "PTS", "REB", "AST", "FG3M", "FG3A", "FGM", "FGA", "FTM", "FTA", "STL", "BLK", "TOV"]

player_avgs = (
    df.groupby(["PLAYER_ID", "PLAYER_NAME"], as_index=False)[STAT_COLS]
      .mean(numeric_only=True)
)

display(player_avgs.sort_values("PTS", ascending=False))


,PLAYER_ID,PLAYER_NAME,MIN,PTS,REB,AST,FG3M,FG3A,FGM,FGA,FTM,FTA,STL,BLK,TOV
9,1628983,Shai Gilgeous-Alexander,33.096667,32.166667,4.833333,6.500000,1.833333,5.500000,11.833333,19.666667,6.666667,8.166667,1.500000,1.333333,2.000000
16,1630224,Jalen Green,22.981667,29.000000,3.000000,3.000000,6.000000,13.000000,10.000000,20.000000,3.000000,5.000000,2.000000,0.000000,2.000000
13,1630166,Deni Avdija,36.515208,27.500000,8.125000,7.125000,2.125000,6.375000,7.625000,15.750000,10.125000,12.375000,0.500000,0.250000,4.875000
20,1631096,Chet Holmgren,31.034167,24.166667,8.000000,1.833333,2.500000,4.500000,8.666667,13.500000,4.333333,4.833333,0.833333,1.666667,3.000000
2,1626164,Devin Booker,34.811905,23.857143,3.714286,6.000000,1.428571,5.000000,8.571429,18.857143,5.285714,5.714286,0.857143,0.428571,3.714286
6,1628415,Dillon Brooks,31.604048,22.571429,3.571429,2.142857,2.142857,6.000000,8.571429,17.714286,3.285714,3.714286,1.571429,0.000000,2.000000
21,1631101,Shaedon Sharpe,32.762500,21.250000,4.625000,2.750000,1.375000,5.875000,8.125000,19.750000,3.625000,5.000000,1.875000,0.125000,3.375000
1,203924,Jerami Grant,31.493333,18.800000,3.800000,3.000000,2.600000,6.000000,5.600000,12.600000,5.000000,6.000000,0.800000,0.800000,1.400000
26,1631123,Jamaree Bouyea,16.750833,18.000000,2.500000,2.500000,2.000000,3.000000,7.000000,11.500000,2.000000,2.000000,0.500000,0.000000,0.000000
24,1631114,Jalen Williams,31.227778,18.000000,4.666667,6.000000,0.666667,1.333333,6.666667,13.666667,4.000000,4.666667,1.666667,0.333333,2.000000


In [5]:
player_avgs = (
    df.groupby(["PLAYER_ID", "PLAYER_NAME"])
      .agg(
          GAMES=("GAME_ID", "nunique"),
          **{c: (c, "mean") for c in STAT_COLS}
      )
      .reset_index()
)

display(player_avgs.sort_values(["GAMES","PTS"], ascending=[False, False]))


,PLAYER_ID,PLAYER_NAME,GAMES,MIN,PTS,REB,AST,FG3M,FG3A,FGM,FGA,FTM,FTA,STL,BLK,TOV
13,1630166,Deni Avdija,8,36.515208,27.500000,8.125000,7.125000,2.125000,6.375000,7.625000,15.750000,10.125000,12.375000,0.500000,0.250000,4.875000
21,1631101,Shaedon Sharpe,8,32.762500,21.250000,4.625000,2.750000,1.375000,5.875000,8.125000,19.750000,3.625000,5.000000,1.875000,0.125000,3.375000
33,1641739,Toumani Camara,8,33.616667,12.750000,3.750000,2.125000,2.625000,6.500000,4.250000,9.750000,1.625000,2.875000,0.625000,0.625000,1.875000
29,1631221,Collin Gillespie,8,27.833958,12.375000,5.125000,4.625000,2.500000,7.375000,4.375000,11.750000,1.125000,1.375000,1.250000,0.250000,1.625000
3,1626220,Royce O'Neale,8,27.496042,9.875000,4.500000,2.000000,2.875000,8.000000,3.500000,9.375000,0.000000,0.125000,1.000000,0.250000,1.250000
2,1626164,Devin Booker,7,34.811905,23.857143,3.714286,6.000000,1.428571,5.000000,8.571429,18.857143,5.285714,5.714286,0.857143,0.428571,3.714286
6,1628415,Dillon Brooks,7,31.604048,22.571429,3.571429,2.142857,2.142857,6.000000,8.571429,17.714286,3.285714,3.714286,1.571429,0.000000,2.000000
34,1642270,Donovan Clingan,7,26.288810,10.428571,9.571429,2.142857,1.142857,2.857143,3.857143,7.571429,1.571429,2.285714,0.142857,0.857143,0.857143
36,1642346,Ryan Dunn,7,19.939286,7.000000,4.000000,1.142857,0.714286,2.142857,2.571429,5.571429,1.142857,1.428571,1.000000,0.142857,0.714286
9,1628983,Shai Gilgeous-Alexander,6,33.096667,32.166667,4.833333,6.500000,1.833333,5.500000,11.833333,19.666667,6.666667,8.166667,1.500000,1.333333,2.000000


In [6]:
def player_summary(df, stat_cols=STAT_COLS):
    g = df.groupby(["PLAYER_ID", "PLAYER_NAME"])

    out = g.agg(GAMES=("GAME_ID", "nunique")).reset_index()

    # mean
    means = g[stat_cols].mean(numeric_only=True).reset_index()
    means = means.rename(columns={c: f"{c}_MEAN" for c in stat_cols})

    # 25th and 75th percentiles
    q25 = g[stat_cols].quantile(0.25).reset_index().rename(columns={c: f"{c}_Q25" for c in stat_cols})
    q75 = g[stat_cols].quantile(0.75).reset_index().rename(columns={c: f"{c}_Q75" for c in stat_cols})

    # merge
    out = out.merge(means, on=["PLAYER_ID","PLAYER_NAME"])
    out = out.merge(q25, on=["PLAYER_ID","PLAYER_NAME"])
    out = out.merge(q75, on=["PLAYER_ID","PLAYER_NAME"])

    return out

summary = player_summary(df)
display(summary.sort_values(["GAMES","PTS_MEAN"], ascending=[False, False]))


,PLAYER_ID,PLAYER_NAME,GAMES,MIN_MEAN,PTS_MEAN,REB_MEAN,AST_MEAN,FG3M_MEAN,FG3A_MEAN,FGM_MEAN,...,AST_Q75,FG3M_Q75,FG3A_Q75,FGM_Q75,FGA_Q75,FTM_Q75,FTA_Q75,STL_Q75,BLK_Q75,TOV_Q75
13,1630166,Deni Avdija,8,36.515208,27.500000,8.125000,7.125000,2.125000,6.375000,7.625000,...,9.00,3.00,7.00,9.25,16.50,13.25,15.00,1.00,0.25,5.50
21,1631101,Shaedon Sharpe,8,32.762500,21.250000,4.625000,2.750000,1.375000,5.875000,8.125000,...,3.00,2.00,6.25,9.25,21.75,4.25,6.25,2.00,0.00,3.75
33,1641739,Toumani Camara,8,33.616667,12.750000,3.750000,2.125000,2.625000,6.500000,4.250000,...,4.00,3.50,7.75,6.00,12.00,2.25,4.25,1.00,1.25,3.00
29,1631221,Collin Gillespie,8,27.833958,12.375000,5.125000,4.625000,2.500000,7.375000,4.375000,...,5.50,3.25,9.25,5.25,13.00,1.25,2.00,2.00,0.25,2.25
3,1626220,Royce O'Neale,8,27.496042,9.875000,4.500000,2.000000,2.875000,8.000000,3.500000,...,2.25,4.25,10.00,4.25,11.00,0.00,0.00,2.00,0.25,1.25
2,1626164,Devin Booker,7,34.811905,23.857143,3.714286,6.000000,1.428571,5.000000,8.571429,...,7.00,2.00,6.00,10.00,22.50,7.00,7.50,1.00,1.00,5.00
6,1628415,Dillon Brooks,7,31.604048,22.571429,3.571429,2.142857,2.142857,6.000000,8.571429,...,3.00,2.50,7.50,10.50,23.00,4.50,5.00,2.50,0.00,2.50
34,1642270,Donovan Clingan,7,26.288810,10.428571,9.571429,2.142857,1.142857,2.857143,3.857143,...,3.00,2.00,3.00,4.50,8.50,2.50,2.50,0.00,1.00,1.00
36,1642346,Ryan Dunn,7,19.939286,7.000000,4.000000,1.142857,0.714286,2.142857,2.571429,...,2.00,1.50,3.50,3.00,7.00,2.00,2.50,1.50,0.00,1.00
9,1628983,Shai Gilgeous-Alexander,6,33.096667,32.166667,4.833333,6.500000,1.833333,5.500000,11.833333,...,6.00,2.00,6.00,12.75,24.00,7.75,9.00,2.00,2.00,2.75


In [9]:
def projected_line(summary_df, player_name: str, mode: str = "MEAN"):
    mode = mode.upper()  # MEAN, Q25, Q75
    cols = ["MIN", "PTS", "REB", "AST", "FG3M", "FTM", "TOV", "STL", "BLK"]

    row = summary_df[summary_df["PLAYER_NAME"] == player_name].copy()
    if row.empty:
        return row

    keep = ["PLAYER_ID","PLAYER_NAME","GAMES"] + [f"{c}_{mode}" for c in cols if f"{c}_{mode}" in row.columns]
    return row[keep]

display(projected_line(summary, "Collin Gillespie", mode="MEAN"))
display(projected_line(summary, "Collin Gillespie", mode="Q75"))  # bullish
display(projected_line(summary, "Collin Gillespie", mode="Q25"))


,PLAYER_ID,PLAYER_NAME,GAMES,MIN_MEAN,PTS_MEAN,REB_MEAN,AST_MEAN,FG3M_MEAN,FTM_MEAN,TOV_MEAN,STL_MEAN,BLK_MEAN
29,1631221,Collin Gillespie,8,27.833958,12.375,5.125,4.625,2.5,1.125,1.625,1.25,0.25


,PLAYER_ID,PLAYER_NAME,GAMES,MIN_Q75,PTS_Q75,REB_Q75,AST_Q75,FG3M_Q75,FTM_Q75,TOV_Q75,STL_Q75,BLK_Q75
29,1631221,Collin Gillespie,8,29.05,15.25,6.0,5.5,3.25,1.25,2.25,2.0,0.25


,PLAYER_ID,PLAYER_NAME,GAMES,MIN_Q25,PTS_Q25,REB_Q25,AST_Q25,FG3M_Q25,FTM_Q25,TOV_Q25,STL_Q25,BLK_Q25
29,1631221,Collin Gillespie,8,26.233333,12.5,4.5,3.75,1.0,0.0,0.75,0.75,0.0


#### Real architechture here


In [11]:


paths = sorted(glob("df_c*.parquet"))

dfs = []
for p in paths:
    # df_c3.parquet -> 3
    cluster = int(p.split("df_c")[1].split(".")[0])
    tmp = pd.read_parquet(p)
    tmp["CLUSTER"] = cluster
    dfs.append(tmp)

all_logs = pd.concat(dfs, ignore_index=True)
print("Loaded:", len(paths), "files | rows:", len(all_logs))
display(all_logs.head())


Loaded: 6 files | rows: 1014


,SEASON_YEAR,GAME_ID,GAME_DATE,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,MATCHUP,WL,...,BLK,TOV,PF,PFD,PLUS_MINUS,DD2,TD3,OPP_TEAM_ABBREVIATION,OPP_TEAM_ID,CLUSTER
0,2025-26,0022500001,2025-10-21,1630598,Aaron Wiggins,1610612760,OKC,Oklahoma City Thunder,OKC vs. HOU,W,...,0,1,1,0,2,0,0,HOU,1610612745,0
1,2025-26,0022500001,2025-10-21,1628983,Shai Gilgeous-Alexander,1610612760,OKC,Oklahoma City Thunder,OKC vs. HOU,W,...,2,3,2,10,3,0,0,HOU,1610612745,0
2,2025-26,0022500001,2025-10-21,1631096,Chet Holmgren,1610612760,OKC,Oklahoma City Thunder,OKC vs. HOU,W,...,0,3,6,3,-1,0,0,HOU,1610612745,0
3,2025-26,0022500001,2025-10-21,1629652,Luguentz Dort,1610612760,OKC,Oklahoma City Thunder,OKC vs. HOU,W,...,0,1,4,2,2,0,0,HOU,1610612745,0
4,2025-26,0022500001,2025-10-21,1628392,Isaiah Hartenstein,1610612760,OKC,Oklahoma City Thunder,OKC vs. HOU,W,...,0,1,6,5,9,0,0,HOU,1610612745,0


In [12]:
STAT_COLS = ["MIN", "PTS", "REB", "AST", "FG3M", "FG3A", "FTM", "FTA", "STL", "BLK", "TOV"]
alpha = 0.6  # 0.6 means: 60% mean + 40% median

g = all_logs.groupby(["CLUSTER", "PLAYER_ID", "PLAYER_NAME"], as_index=False)

stable = g.agg(
    GAMES=("GAME_ID", "nunique"),
    **{f"{c}_MEAN": (c, "mean") for c in STAT_COLS},
    **{f"{c}_MED":  (c, "median") for c in STAT_COLS},
)

# blend -> BASE
for c in STAT_COLS:
    stable[f"{c}_BASE"] = alpha * stable[f"{c}_MEAN"] + (1 - alpha) * stable[f"{c}_MED"]

# (optional) sort for wrangler
stable = stable.sort_values(["CLUSTER", "GAMES", "PTS_BASE"], ascending=[True, False, False])

display(stable.head(25))



,CLUSTER,PLAYER_ID,PLAYER_NAME,GAMES,MIN_MEAN,PTS_MEAN,REB_MEAN,AST_MEAN,FG3M_MEAN,FG3A_MEAN,...,PTS_BASE,REB_BASE,AST_BASE,FG3M_BASE,FG3A_BASE,FTM_BASE,FTA_BASE,STL_BASE,BLK_BASE,TOV_BASE
13,0,1630166,Deni Avdija,8,36.515208,27.500000,8.125000,7.125000,2.125000,6.375000,...,26.900000,8.075000,7.075000,2.075000,6.625000,9.875000,13.425000,0.500000,0.150000,4.725000
21,0,1631101,Shaedon Sharpe,8,32.762500,21.250000,4.625000,2.750000,1.375000,5.875000,...,20.750000,4.575000,2.850000,1.225000,5.925000,3.775000,5.400000,1.725000,0.075000,3.025000
33,0,1641739,Toumani Camara,8,33.616667,12.750000,3.750000,2.125000,2.625000,6.500000,...,13.650000,3.450000,2.275000,2.575000,6.500000,1.375000,2.525000,0.575000,0.375000,1.925000
29,0,1631221,Collin Gillespie,8,27.833958,12.375000,5.125000,4.625000,2.500000,7.375000,...,12.625000,5.275000,4.575000,2.700000,7.425000,1.075000,1.425000,1.150000,0.150000,1.775000
3,0,1626220,Royce O'Neale,8,27.496042,9.875000,4.500000,2.000000,2.875000,8.000000,...,10.525000,4.500000,1.800000,3.125000,8.000000,0.000000,0.075000,1.000000,0.150000,1.150000
2,0,1626164,Devin Booker,7,34.811905,23.857143,3.714286,6.000000,1.428571,5.000000,...,23.914286,3.428571,6.400000,1.657143,5.000000,5.571429,6.228571,0.914286,0.257143,3.828571
6,0,1628415,Dillon Brooks,7,31.604048,22.571429,3.571429,2.142857,2.142857,6.000000,...,21.942857,3.342857,2.085714,2.085714,6.000000,3.171429,3.828571,1.742857,0.000000,2.000000
34,0,1642270,Donovan Clingan,7,26.288810,10.428571,9.571429,2.142857,1.142857,2.857143,...,10.257143,9.742857,2.085714,1.085714,2.914286,1.342857,2.171429,0.085714,0.914286,0.914286
36,0,1642346,Ryan Dunn,7,19.939286,7.000000,4.000000,1.142857,0.714286,2.142857,...,6.600000,4.000000,0.685714,0.428571,1.685714,1.085714,1.657143,1.000000,0.085714,0.828571
9,0,1628983,Shai Gilgeous-Alexander,6,33.096667,32.166667,4.833333,6.500000,1.833333,5.500000,...,31.700000,4.900000,6.100000,1.900000,5.500000,6.800000,8.100000,1.700000,1.400000,2.000000


In [ ]:
stable

In [13]:
g = all_logs.groupby(["CLUSTER", "PLAYER_ID", "PLAYER_NAME"], as_index=False)

longshots = g.agg(
    GAMES=("GAME_ID", "nunique"),
    **{f"{c}_Q75": (c, lambda s: s.quantile(0.75)) for c in STAT_COLS},
)

# optional: sort by the upside stat you care about
longshots = longshots.sort_values(["CLUSTER", "GAMES", "PTS_Q75"], ascending=[True, False, False])

display(longshots.head(25))


,CLUSTER,PLAYER_ID,PLAYER_NAME,GAMES,MIN_Q75,PTS_Q75,REB_Q75,AST_Q75,FG3M_Q75,FG3A_Q75,FTM_Q75,FTA_Q75,STL_Q75,BLK_Q75,TOV_Q75
13,0,1630166,Deni Avdija,8,38.968333,30.50,9.00,9.00,3.00,7.00,13.25,15.00,1.00,0.25,5.50
21,0,1631101,Shaedon Sharpe,8,34.483333,23.50,5.75,3.00,2.00,6.25,4.25,6.25,2.00,0.00,3.75
33,0,1641739,Toumani Camara,8,36.501667,17.75,6.00,4.00,3.50,7.75,2.25,4.25,1.00,1.25,3.00
29,0,1631221,Collin Gillespie,8,29.050000,15.25,6.00,5.50,3.25,9.25,1.25,2.00,2.00,0.25,2.25
3,0,1626220,Royce O'Neale,8,29.262500,12.75,6.00,2.25,4.25,10.00,0.00,0.00,2.00,0.25,1.25
2,0,1626164,Devin Booker,7,37.583333,27.00,4.50,7.00,2.00,6.00,7.00,7.50,1.00,1.00,5.00
6,0,1628415,Dillon Brooks,7,34.366667,26.00,4.00,3.00,2.50,7.50,4.50,5.00,2.50,0.00,2.50
34,0,1642270,Donovan Clingan,7,28.192500,11.50,10.50,3.00,2.00,3.00,2.50,2.50,0.00,1.00,1.00
36,0,1642346,Ryan Dunn,7,21.824167,8.00,5.00,2.00,1.50,3.50,2.00,2.50,1.50,0.00,1.00
9,0,1628983,Shai Gilgeous-Alexander,6,34.175000,34.25,5.75,6.00,2.00,6.00,7.75,9.00,2.00,2.00,2.75


In [ ]:
longshots

In [14]:
MIN_GAMES = 3

stable_f = stable[stable["GAMES"] >= MIN_GAMES].copy()
longshots_f = longshots[longshots["GAMES"] >= MIN_GAMES].copy()

print("Stable rows:", len(stable_f), " | Longshot rows:", len(longshots_f))


Stable rows: 135  | Longshot rows: 135


In [ ]:
longshots_f

In [15]:
stable_f

,CLUSTER,PLAYER_ID,PLAYER_NAME,GAMES,MIN_MEAN,PTS_MEAN,REB_MEAN,AST_MEAN,FG3M_MEAN,FG3A_MEAN,...,PTS_BASE,REB_BASE,AST_BASE,FG3M_BASE,FG3A_BASE,FTM_BASE,FTA_BASE,STL_BASE,BLK_BASE,TOV_BASE
13,0,1630166,Deni Avdija,8,36.515208,27.500000,8.125000,7.125000,2.125,6.375000,...,26.900,8.075,7.075,2.075,6.625,9.875,13.425,0.500,0.150,4.725
21,0,1631101,Shaedon Sharpe,8,32.762500,21.250000,4.625000,2.750000,1.375,5.875000,...,20.750,4.575,2.850,1.225,5.925,3.775,5.400,1.725,0.075,3.025
33,0,1641739,Toumani Camara,8,33.616667,12.750000,3.750000,2.125000,2.625,6.500000,...,13.650,3.450,2.275,2.575,6.500,1.375,2.525,0.575,0.375,1.925
29,0,1631221,Collin Gillespie,8,27.833958,12.375000,5.125000,4.625000,2.500,7.375000,...,12.625,5.275,4.575,2.700,7.425,1.075,1.425,1.150,0.150,1.775
3,0,1626220,Royce O'Neale,8,27.496042,9.875000,4.500000,2.000000,2.875,8.000000,...,10.525,4.500,1.800,3.125,8.000,0.000,0.075,1.000,0.150,1.150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,4,202691,Klay Thompson,5,23.722667,14.200000,3.200000,1.800000,3.800,9.000000,...,13.320,3.120,1.480,3.880,9.000,0.480,0.720,0.760,0.120,1.240
187,4,1629023,P.J. Washington,4,35.777917,18.000000,10.500000,0.500000,2.250,4.000000,...,18.600,10.300,0.500,1.950,3.600,3.050,4.900,0.500,2.000,3.350
191,4,1631108,Max Christie,4,29.804167,14.750000,3.750000,2.000000,2.500,5.750000,...,14.450,3.650,2.000,2.500,5.850,3.050,3.050,0.500,0.500,0.500
194,4,1642948,Ryan Nembhard,3,28.233333,8.333333,3.000000,7.666667,1.000,4.666667,...,7.800,3.000,7.800,1.000,4.400,0.000,0.000,0.800,0.000,4.000
